# 06 - Network Analysis of Mitochondrial Genomes from FASTA Data

This notebook focuses on constructing and analyzing a similarity network of mitochondrial genomes based *solely* on features extractable from FASTA sequence files. This approach allows us to understand relationships and patterns between genomes using fundamental sequence characteristics.

## Analysis Steps:
1.  **Data Loading & Feature Extraction:** Read mitochondrial genome sequences from a FASTA file and extract basic features (length, GC content, nucleotide counts, base skews, dinucleotide frequencies).
2.  **Feature Scaling:** Standardize the extracted numerical features to ensure all contribute equally to similarity calculations.
3.  **Similarity Matrix Calculation:** Compute pairwise similarity (e.g., cosine similarity) between all genomes based on their feature vectors.
4.  **Network Construction:** Build a graph where genomes are nodes and edges represent a high degree of similarity between them.
5.  **Interactive Network Visualization:** Create a dynamic network graph using Plotly, allowing for interactive exploration including hovering over nodes to see detailed information.
6.  **Network Statistics & Interpretation:** Calculate and discuss key network properties (e.g., density, connectivity, central nodes) and provide biological interpretations of the observed genomic relationships.

**Author:** yashmgupta  
**Date:** 2025-06-19

In [ ]:
# Cell 1: Import required libraries
from Bio import SeqIO
from Bio.SeqUtils import gc_fraction
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import plotly.graph_objects as go
import plotly.offline as pyo
import seaborn as sns
import warnings
import sys
import subprocess

try:
    import community as co # For community detection
except ModuleNotFoundError:
    print("'community' module not found. Attempting to install 'python-louvain'...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "python-louvain"])
        import community as co
        print("'python-louvain' installed successfully and 'community' module imported.")
    except Exception as e:
        print(f"Failed to install 'python-louvain'. Please install it manually: pip install python-louvain. Error: {e}")
        # Re-raise to prevent further execution if the module is critical
        raise


# Initialize Plotly for offline use
pyo.init_notebook_mode(connected=True)

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("All necessary libraries imported.")

In [ ]:
# Assuming "All_orthomt.fasta" is available in the environment
input_fasta_all = "All_orthomt.fasta"
try:
    records_all = list(SeqIO.parse(input_fasta_all, "fasta"))
    print(f"Total sequences loaded from '{input_fasta_all}': {len(records_all)}")

    # Cell 3: Separate UNVERIFIED and VERIFIED records
    unverified_records = [rec for rec in records_all if "UNVERIFIED" in rec.description]
    verified_records   = [rec for rec in records_all if "UNVERIFIED" not in rec.description]

    print(f"UNVERIFIED sequences: {len(unverified_records)}")
    print(f"VERIFIED sequences: {len(verified_records)}")

    # Cell 4: Save VERIFIED records to new FASTA
    fasta_file = "verified_only.fasta" # This will be the input for network analysis
    SeqIO.write(verified_records, fasta_file, "fasta")
    print(f"Verified-only FASTA saved to: {fasta_file}")

    # Cell 5: Save UNVERIFIED record IDs and descriptions to CSV
    unverified_list = [{"id": rec.id, "description": rec.description} for rec in unverified_records]
    df_unverified = pd.DataFrame(unverified_list)
    csv_output_unverified = "unverified_sequences.csv"
    df_unverified.to_csv(csv_output_unverified, index=False)
    print(f"UNVERIFIED sequence list saved to: {csv_output_unverified}")

    # Cell 6: Save UNVERIFIED records to new FASTA (as requested previously)
    output_unverified_fasta = "unverified_only.fasta"
    SeqIO.write(unverified_records, output_unverified_fasta, "fasta")
    print(f"Unverified-only FASTA saved to: {output_unverified_fasta}")

except FileNotFoundError:
    print(f"Error: Input FASTA file '{input_fasta_all}' not found. Please ensure it's in the correct directory.")
    exit() # Exit if the primary input file is missing
except Exception as e:
    print(f"An error occurred during initial FASTA processing: {e}")
    exit()

In [ ]:
similarity_threshold = 0.85 # Minimum cosine similarity for an edge

print("\n--- Starting Network Analysis ---")
print(f"Using FASTA file: {fasta_file}")
print(f"Similarity threshold: {similarity_threshold}")

# Cell 8: Data Loading & Feature Extraction for Network Analysis
print("\n--- Extracting features from sequences ---")

def calculate_base_skew(sequence):
    """Calculates GC and AT skew for a given sequence."""
    g = sequence.count('G')
    c = sequence.count('C')
    a = sequence.count('A')
    t = sequence.count('T')

    gc_skew = (g - c) / (g + c) if (g + c) > 0 else 0
    at_skew = (a - t) / (a + t) if (a + t) > 0 else 0
    return gc_skew, at_skew

def calculate_dinucleotide_frequencies(sequence):
    """Calculates the frequencies of all 16 dinucleotides."""
    dinucleotides = defaultdict(int)
    total_dinucleotides = 0
    for i in range(len(sequence) - 1):
        dinucleotide = sequence[i:i+2].upper()
        if all(base in 'ATGC' for base in dinucleotide):
            dinucleotides[dinucleotide] += 1
            total_dinucleotides += 1

    frequencies = {
        dino: count / total_dinucleotides if total_dinucleotides > 0 else 0
        for dino, count in dinucleotides.items()
    }
    # Ensure all 16 dinucleotides are present, even if frequency is 0
    all_dinucleotides = [a + b for a in 'ATGC' for b in 'ATGC']
    for dino in all_dinucleotides:
        if dino not in frequencies:
            frequencies[dino] = 0
    return frequencies

extracted_features = []
try:
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence_id = record.id
        full_description = record.description
        sequence = str(record.seq).upper()

        # Attempt to parse organism name from description
        organism = "Unknown"
        parts = full_description.split(' ')
        if len(parts) > 1:
            # Common pattern: "Accession_ID SpeciesName [mitochondrion] other_info"
            # Try to get the first two words as species name, then refine
            organism_guess = " ".join(parts[1:3]).replace("mitochondrion", "").strip()
            # Remove any trailing bracketed text, common for taxonomy
            organism = organism_guess.split('[')[0].strip()
            # Basic capitalization for consistency
            if organism and len(organism.split(' ')) > 1:
                organism = ' '.join([word.capitalize() if i == 0 else word for i, word in enumerate(organism.split(' '))])
            elif organism:
                 organism = organism.capitalize()
            # Fallback if organism is still empty or too short
            if not organism or len(organism) < 3:
                organism = "Unknown"


        seq_len = len(sequence)
        gc_cont = gc_fraction(sequence) if seq_len > 0 else 0
        
        a_count = sequence.count('A')
        t_count = sequence.count('T')
        g_count = sequence.count('G')
        c_count = sequence.count('C')

        gc_skew, at_skew = calculate_base_skew(sequence)
        dinucleotide_freqs = calculate_dinucleotide_frequencies(sequence)

        row = {
            "sequence_id": sequence_id,
            "organism": organism,
            "full_info": full_description, # Keep full info for hover text
            "length": seq_len,
            "gc_content": gc_cont,
            "A_count": a_count,
            "T_count": t_count,
            "G_count": g_count,
            "C_count": c_count,
            "GC_skew": gc_skew,
            "AT_skew": at_skew,
            **dinucleotide_freqs # Unpack dinucleotide frequencies
        }
        extracted_features.append(row)

    df_features = pd.DataFrame(extracted_features)

    # Handle potential duplicate sequence_ids
    if df_features['sequence_id'].duplicated().any():
        warnings.warn("Duplicate sequence_ids found. Keeping the first occurrence.")
        df_features.drop_duplicates(subset='sequence_id', keep='first', inplace=True)

    df_features.set_index("sequence_id", inplace=True)

    print(f"Extracted features for {len(df_features)} sequences.")
    print("First 5 rows of extracted features:")
    print(df_features.head())

except FileNotFoundError:
    print(f"Error: Processed FASTA file '{fasta_file}' not found. Please ensure it exists after initial processing.")
    exit()
except Exception as e:
    print(f"An error occurred during feature extraction: {e}")
    exit()

In [ ]:
print("\n--- Scaling numerical features ---")

# Separate numerical features from descriptive columns
X = df_features.drop(columns=['organism', 'full_info'])
y = df_features['organism'] # Keep organism for coloring/analysis

# Initialize StandardScaler
scaler = StandardScaler()

# Fit and transform the numerical features
X_scaled = scaler.fit_transform(X)

# Create a new DataFrame with scaled features
df_scaled_features = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

print(f"Data after feature engineering: {df_scaled_features.shape[0]} sequences, {df_scaled_features.shape[1]} features.")
print("First 5 rows of scaled features:")
print(df_scaled_features.head())

In [ ]:
# Cell 10: Similarity Matrix Calculation and Network Construction
print("\n--- Calculating similarity matrix and constructing network ---")

# Calculate cosine similarity between all pairs of scaled feature vectors
similarity_matrix = cosine_similarity(X_scaled)

# Convert to DataFrame for easier inspection, using sequence_ids as index/columns
similarity_df = pd.DataFrame(similarity_matrix, index=df_scaled_features.index, columns=df_scaled_features.index)

# Initialize a networkx graph
network = nx.Graph()

# Add nodes with attributes (organism, full_info)
for seq_id in df_features.index:
    network.add_node(seq_id, 
                     organism=df_features.loc[seq_id, 'organism'], 
                     full_info=df_features.loc[seq_id, 'full_info'])

# Add edges based on similarity threshold
edges_added_count = 0
for i in range(len(similarity_df.index)):
    for j in range(i + 1, len(similarity_df.columns)): # Avoid self-loops and duplicate edges
        seq_id1 = similarity_df.index[i]
        seq_id2 = similarity_df.columns[j]
        similarity = similarity_df.iloc[i, j]

        if similarity >= similarity_threshold:
            network.add_edge(seq_id1, seq_id2, weight=similarity)
            edges_added_count += 1

print(f"Network created with {network.number_of_nodes()} nodes and {network.number_of_edges()} edges.")
print(f"Network density: {nx.density(network):.4f}")
print(f"Number of connected components: {nx.number_connected_components(network)}")

In [ ]:
# Cell 11: Interactive Network Visualization (Plotly)
print("\n--- Generating interactive network visualization ---")

if network.number_of_edges() == 0:
    print("No edges in the network. Skipping network visualization.")
    # Ensure 'pos' is not defined if visualization is skipped to avoid errors later
    pos = None 
else:
    # Use a force-directed layout for node positioning
    # Adjust k (optimal distance between nodes) and iterations for better layouts
    pos = nx.spring_layout(network, k=0.3, iterations=50, dim=2) 

    # Prepare data for Plotly
    edge_x = []
    edge_y = []
    edge_weights = []
    for edge in network.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None]) # 'None' creates a break in the line, preventing drawing between disconnected segments
        edge_y.extend([y0, y1, None])
        edge_weights.append(network.edges[edge]['weight'])

    node_x = [pos[node][0] for node in network.nodes()]
    node_y = [pos[node][1] for node in network.nodes()]

    # Get node information for coloring and hover
    unique_organisms = y.unique()
    # Generate a discrete color palette using seaborn
    palette = sns.color_palette("tab10", n_colors=len(unique_organisms)).as_hex()
    organism_color_map = {organism: palette[i] for i, organism in enumerate(unique_organisms)}
    
    node_colors = [organism_color_map.get(network.nodes[node]['organism'], '#CCCCCC') for node in network.nodes()] # Default color for unknown
    node_hover_texts = [network.nodes[node]['full_info'] for node in network.nodes()]

    # Create edge trace
    edge_trace = go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines',
        name='Edges' # Added name for clarity if debugging legend
    )

    # Create node trace
    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=False, # No continuous colorscale needed for discrete colors
            color=node_colors, # Direct hex/RGB colors
            size=10,
            line_width=1),
        text=node_hover_texts,
        name='Nodes' # Added name for clarity if debugging legend
    )

    # Create a mapping for legend based on organism names
    legend_points = []
    for organism_name, color_hex in organism_color_map.items():
        legend_points.append(go.Scatter(
            x=[None], y=[None], # Dummy points
            mode='markers',
            marker=dict(size=10, color=color_hex, symbol='circle'), # Added symbol for consistent look
            name=organism_name, # Use organism name as legend entry
            hoverinfo='none',
            showlegend=True # Ensure these dummy points show in legend
        ))

    fig = go.Figure(data=[edge_trace, node_trace] + legend_points,
                    layout=go.Layout(
                        title=dict(
                            text='Mitochondrial Genome Similarity Network',
                            font=dict(size=16) 
                        ),
                        showlegend=True,
                        hovermode='closest',
                        margin=dict(b=20,l=5,r=5,t=40),
                        annotations=[ dict(
                            text="Network analysis based on FASTA-derived features",
                            showarrow=False,
                            xref="paper", yref="paper",
                            x=0.005, y=-0.002 ) ],
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False, mirror=True),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, mirror=True),
                        height=800,
                        template="plotly_white" # Use a clean white background template
                    ))

    pyo.iplot(fig) # Display the interactive plot
    html_output_path = "06_interactive_network_plot.html"
    fig.write_html(html_output_path) # Save as HTML for external viewing
    print(f"Interactive network plot saved as {html_output_path}")

In [ ]:
# Cell 12: Save Network Analysis Results to CSV (with centrality and clustering)
print("\n--- Saving network data to CSV files ---")

# 12.1 Calculate centrality measures and prepare node data
node_data = []

if network.number_of_nodes() > 0:
    print("Calculating node centrality measures...")
    try:
        degree_centrality = nx.degree_centrality(network)
    except Exception as e:
        print(f"Could not calculate degree centrality: {e}")
        degree_centrality = {node: 0 for node in network.nodes()} # Default to 0
    
    try:
        # Betweenness centrality can be computationally intensive for large graphs
        betweenness_centrality = nx.betweenness_centrality(network)
    except Exception as e:
        print(f"Could not calculate betweenness centrality: {e}")
        betweenness_centrality = {node: 0 for node in network.nodes()} # Default to 0

    try:
        closeness_centrality = nx.closeness_centrality(network)
    except Exception as e:
        print(f"Could not calculate closeness centrality: {e}")
        closeness_centrality = {node: 0 for node in network.nodes()} # Default to 0

else:
    print("Network has no nodes, skipping centrality calculations.")
    degree_centrality = {}
    betweenness_centrality = {}
    closeness_centrality = {}


for node_id, attributes in network.nodes(data=True):
    node_info = {
        "node_id": node_id,
        "organism": attributes.get("organism", "N/A"),
        "full_info": attributes.get("full_info", "N/A"),
        "x_pos": pos[node_id][0] if pos and node_id in pos else None, # Check if pos exists
        "y_pos": pos[node_id][1] if pos and node_id in pos else None, # Check if pos exists
        "degree_centrality": degree_centrality.get(node_id, 0),
        "betweenness_centrality": betweenness_centrality.get(node_id, 0),
        "closeness_centrality": closeness_centrality.get(node_id, 0)
    }
    node_data.append(node_info)

df_nodes = pd.DataFrame(node_data)
output_nodes_csv = "network_nodes.csv"
df_nodes.to_csv(output_nodes_csv, index=False)
print(f"Network node data saved to: {output_nodes_csv}")

# 12.2 Save Edge Data to CSV
edge_data = []
for u, v, attributes in network.edges(data=True):
    edge_info = {
        "source": u,
        "target": v,
        "weight": attributes.get("weight", 0) # Default to 0 if weight not found
    }
    edge_data.append(edge_info)

df_edges = pd.DataFrame(edge_data)
output_edges_csv = "network_edges.csv"
df_edges.to_csv(output_edges_csv, index=False)
print(f"Network edge data saved to: {output_edges_csv}")

In [ ]:
# Cell 13: Community Detection and Representative Species
print("\n--- Performing Community Detection and Identifying Representative Species ---")

cluster_results = []
# Ensure network has enough nodes and edges for meaningful community detection
if network.number_of_nodes() > 1 and network.number_of_edges() > 0:
    try:
        # Compute the best partition using the Louvain method
        partition = co.best_partition(network)
        num_communities = len(set(partition.values()))
        print(f"Found {num_communities} communities.")

        # Add community/cluster ID to node data
        # Ensure 'cluster_id' column is added correctly
        df_nodes['cluster_id'] = df_nodes['node_id'].map(partition).fillna(-1).astype(int)

        # Identify representative species for each cluster
        cluster_representatives = {}
        for cluster_id in sorted(set(partition.values())):
            nodes_in_cluster = [node for node, cid in partition.items() if cid == cluster_id]
            
            # Get organisms for nodes in this cluster from the network's node attributes
            organisms_in_cluster = []
            for node_id in nodes_in_cluster:
                organism = network.nodes[node_id].get('organism')
                if organism and organism != "N/A" and organism != "Unknown": # Filter out default/unknown organisms
                    organisms_in_cluster.append(organism)
            
            if organisms_in_cluster:
                # Find the most common organism in the cluster
                organism_counts = Counter(organisms_in_cluster)
                most_common_organism = organism_counts.most_common(1)[0][0]
                cluster_representatives[cluster_id] = most_common_organism
            else:
                cluster_representatives[cluster_id] = "N/A (No identifiable organism)"

        # Prepare cluster summary data
        for cluster_id, representative in cluster_representatives.items():
            cluster_results.append({
                "cluster_id": cluster_id,
                "representative_organism": representative,
                "num_nodes_in_cluster": list(partition.values()).count(cluster_id)
            })

    except Exception as e:
        print(f"Error during community detection or representative species identification: {e}")
        print("Please ensure 'python-louvain' is installed (e.g., pip install python-louvain).")
        # Ensure 'cluster_id' column is added even if clustering fails for consistency
        if 'cluster_id' not in df_nodes.columns:
            df_nodes['cluster_id'] = -1 # Assign a default invalid cluster ID
else:
    print("Network is too small or has no edges for meaningful community detection.")
    if 'cluster_id' not in df_nodes.columns:
        df_nodes['cluster_id'] = -1 # Assign a default invalid cluster ID

# Save updated node data with cluster IDs
output_nodes_clustered_csv = "network_nodes_clustered.csv"
df_nodes.to_csv(output_nodes_clustered_csv, index=False)
print(f"Updated network node data with cluster IDs saved to: {output_nodes_clustered_csv}")

# Save cluster representatives to CSV
if cluster_results:
    df_clusters = pd.DataFrame(cluster_results)
    output_clusters_csv = "network_clusters_summary.csv"
    df_clusters.to_csv(output_clusters_csv, index=False)
    print(f"Cluster summary and representative species saved to: {output_clusters_csv}")
else:
    print("No cluster summary to save as no clusters were identified.")

In [ ]:
# Cell 14: Visualize Network with Communities
print("\n--- Visualizing Network with Communities and Representative Species ---")

# --- NEW: Configuration for highlighting ---
# Set the cluster ID you want to highlight. Set to None to show all clusters colored.
# You can get cluster IDs from the 'network_clusters_summary.csv' file or the output of Cell 13.
highlight_cluster_id = None # Example: Set to an integer like 0, 1, 2, etc., to highlight a specific cluster.


if network.number_of_edges() == 0 or not network.nodes() or not 'cluster_id' in df_nodes.columns:
    print("Cannot visualize communities: Network has no edges, no nodes, or clustering was not performed/failed.")
else:
    # Ensure pos exists from previous visualization, if not, generate it
    # This block is important as 'pos' is used for all node coordinates.
    if 'pos' not in locals() or pos is None:
        print("Node positions not found. Generating new spring layout for visualization.")
        pos = nx.spring_layout(network, k=0.3, iterations=50, dim=2)

    # Re-create edge trace (similar to previous visualization)
    edge_x = []
    edge_y = []
    for edge in network.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace_cluster = go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines',
        name='Edges',
        showlegend=False # Edges don't need a legend entry for this view
    )

    # Get node information for coloring by cluster
    unique_cluster_ids = df_nodes['cluster_id'].unique()
    # Generate a discrete color palette
    cluster_palette = sns.color_palette("hls", n_colors=len(unique_cluster_ids)).as_hex()
    # Create a mapping from cluster ID to color
    cluster_color_map = {cluster_id: cluster_palette[i] for i, cluster_id in enumerate(sorted(unique_cluster_ids))}
    
    # --- NEW LOGIC: Conditional creation of node traces for highlighting ---
    node_traces = []
    
    if highlight_cluster_id is not None:
        # Mode 1: Highlight a specific cluster
        nodes_for_highlighted_trace_x = []
        nodes_for_highlighted_trace_y = []
        colors_for_highlighted_trace = []
        hover_texts_for_highlighted_trace = []

        # Nodes not in the highlighted cluster will be gray
        nodes_for_gray_trace_x = []
        nodes_for_gray_trace_y = []
        hover_texts_for_gray_trace = []
        
        # Determine representative organism for the highlighted cluster for legend
        highlight_rep_org = "N/A"
        for cr in cluster_results:
            if cr['cluster_id'] == highlight_cluster_id:
                highlight_rep_org = cr['representative_organism']
                break

        for node_id in network.nodes():
            if node_id in df_nodes.index:
                cluster_id_for_node = df_nodes.loc[node_id, 'cluster_id']
                organism_for_node = network.nodes[node_id].get('organism', 'N/A')
                full_info_for_node = network.nodes[node_id].get('full_info', 'N/A')

                hover_text = f"{full_info_for_node}<br>Cluster: {cluster_id_for_node}<br>Organism: {organism_for_node}"
                
                if cluster_id_for_node == highlight_cluster_id:
                    nodes_for_highlighted_trace_x.append(pos[node_id][0])
                    nodes_for_highlighted_trace_y.append(pos[node_id][1])
                    colors_for_highlighted_trace.append(cluster_color_map.get(cluster_id_for_node, '#CCCCCC'))
                    hover_texts_for_highlighted_trace.append(hover_text)
                else:
                    nodes_for_gray_trace_x.append(pos[node_id][0])
                    nodes_for_gray_trace_y.append(pos[node_id][1])
                    hover_texts_for_gray_trace.append(hover_text)
            else:
                # Fallback for nodes in network but not in df_nodes (should ideally not happen)
                nodes_for_gray_trace_x.append(pos[node_id][0])
                nodes_for_gray_trace_y.append(pos[node_id][1])
                hover_texts_for_gray_trace.append(f"{node_id}<br>Cluster: N/A<br>Organism: N/A (Data Missing)")

        # Add trace for the highlighted cluster
        if nodes_for_highlighted_trace_x:
            node_traces.append(
                go.Scatter(
                    x=nodes_for_highlighted_trace_x,
                    y=nodes_for_highlighted_trace_y,
                    mode='markers',
                    hoverinfo='text',
                    marker=dict(
                        showscale=False,
                        color=colors_for_highlighted_trace,
                        size=12,
                        line=dict(color='Black', width=0.5),
                        line_width=1
                    ),
                    text=hover_texts_for_highlighted_trace,
                    name=f"Highlighted: Cluster {highlight_cluster_id} ({highlight_rep_org})",
                    showlegend=True
                )
            )
        
        # Add trace for all other nodes (in gray)
        if nodes_for_gray_trace_x:
            node_traces.append(
                go.Scatter(
                    x=nodes_for_gray_trace_x,
                    y=nodes_for_gray_trace_y,
                    mode='markers',
                    hoverinfo='text',
                    marker=dict(
                        showscale=False,
                        color='#CCCCCC', # Gray for non-highlighted
                        size=12,
                        line=dict(color='Black', width=0.5),
                        line_width=1
                    ),
                    text=hover_texts_for_gray_trace,
                    name="Other Nodes (Gray)",
                    showlegend=True # Show this in legend so user knows what gray means
                )
            )

    else: 
        # Mode 2: Show all clusters with separate traces for interactive toggling
        for cluster_id in sorted(unique_cluster_ids):
            # Filter nodes for the current cluster
            cluster_nodes = [node_id for node_id in network.nodes() if node_id in df_nodes.index and df_nodes.loc[node_id, 'cluster_id'] == cluster_id]
            
            # Prepare data for this cluster's trace
            cluster_node_x = [pos[node][0] for node in cluster_nodes]
            cluster_node_y = [pos[node][1] for node in cluster_nodes]
            cluster_node_hover_texts = [
                f"{network.nodes[node]['full_info']}<br>Cluster: {cluster_id}<br>Organism: {network.nodes[node]['organism']}"
                for node in cluster_nodes
            ]

            representative_org = "N/A"
            for cr in cluster_results:
                if cr['cluster_id'] == cluster_id:
                    representative_org = cr['representative_organism']
                    break

            node_trace_cluster_i = go.Scatter(
                x=cluster_node_x,
                y=cluster_node_y,
                mode='markers',
                hoverinfo='text',
                marker=dict(
                    showscale=False,
                    color=cluster_color_map.get(cluster_id, '#CCCCCC'), # Use specific cluster color
                    size=12,
                    line=dict(color='Black', width=0.5),
                    line_width=1
                ),
                text=cluster_node_hover_texts,
                name=f"Cluster {cluster_id} ({representative_org})", # Name for legend entry
                showlegend=True # Ensure this trace appears in the legend
            )
            node_traces.append(node_trace_cluster_i)

    # Combine edges and all node traces
    fig_cluster = go.Figure(data=[edge_trace_cluster] + node_traces,
                            layout=go.Layout(
                                title=dict(
                                    text='Mitochondrial Genome Network with Detected Communities',
                                    font=dict(size=16)
                                ),
                                showlegend=True, # Ensure legend is shown
                                hovermode='closest',
                                margin=dict(b=20,l=5,r=5,t=40),
                                annotations=[ dict(
                                    text="Nodes colored by detected community. Click legend to toggle visibility. <br>Set 'highlight_cluster_id' to focus on a specific cluster.",
                                    showarrow=False,
                                    xref="paper", yref="paper",
                                    x=0.005, y=-0.002 ) ],
                                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False, mirror=True),
                                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, mirror=True),
                                height=800,
                                template="plotly_white"
                            ))

    pyo.iplot(fig_cluster)
    html_output_path_cluster_plot = "07_interactive_community_network_plot.html"
    fig_cluster.write_html(html_output_path_cluster_plot)
    print(f"Interactive community network plot saved as {html_output_path_cluster_plot}")

In [ ]:
# Cell 14: Network Statistics & Interpretation Report
print("\n--- Generating final network analysis report ---")

report_lines = []
report_lines.append("="*70)
report_lines.append("Mitochondrial Genome Network Analysis Report")
report_lines.append("="*70)
report_lines.append(f"Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")
report_lines.append(f"Input FASTA for analysis: {fasta_file}")
report_lines.append(f"Similarity Threshold for Edges: {similarity_threshold}")
report_lines.append("")

if network.number_of_nodes() > 0:
    report_lines.append(f"Total Nodes (Mitochondrial Genomes): {network.number_of_nodes()}")
    report_lines.append(f"Total Edges (Similarities > {similarity_threshold}): {network.number_of_edges()}")
    report_lines.append(f"Network Density: {nx.density(network):.4f}")
    report_lines.append(f"Number of Connected Components: {nx.number_connected_components(network)}")
    report_lines.append("")

    if network.number_of_edges() > 0:
        report_lines.append("Top 10 Nodes by Degree Centrality:")
        degree_centrality = nx.degree_centrality(network)
        sorted_degrees = sorted(degree_centrality.items(), key=lambda item: item[1], reverse=True)
        for i, (node, degree_val) in enumerate(sorted_degrees[:10]):
            organism = network.nodes[node].get('organism', 'N/A')
            report_lines.append(f"  {i+1}. {node} (Organism: {organism}, Degree Centrality: {degree_val:.4f})")
        report_lines.append("")

        report_lines.append("Nodes with Zero Degree (Isolated Nodes):")
        isolated_nodes = [node for node, degree in network.degree() if degree == 0]
        if isolated_nodes:
            for node in isolated_nodes:
                organism = network.nodes[node].get('organism', 'N/A')
                report_lines.append(f"  - {node} (Organism: {organism})")
        else:
            report_lines.append("  No isolated nodes found.")
        report_lines.append("")

        report_lines.append("Community Detection Summary:")
        if cluster_results:
            for cluster_info in cluster_results:
                report_lines.append(f"  - Cluster {cluster_info['cluster_id']}:")
                report_lines.append(f"    - Representative Organism: {cluster_info['representative_organism']}")
                report_lines.append(f"    - Number of Nodes: {cluster_info['num_nodes_in_cluster']}")
        else:
            report_lines.append("  Community detection was not performed or yielded no clusters.")
        report_lines.append("")

        report_lines.append("Output Files Generated:")
        report_lines.append(f"  - Interactive Network Plot: {html_output_path}")
        report_lines.append(f"  - Network Node Data (with centrality): {output_nodes_csv}")
        report_lines.append(f"  - Network Edge Data: {output_edges_csv}")
        report_lines.append(f"  - Network Node Data (with cluster IDs): {output_nodes_clustered_csv}")
        report_lines.append(f"  - Cluster Summary: {output_clusters_csv}")
        report_lines.append("")

        report_lines.append("Suggestions for Further Analysis:")
        report_lines.append("  - Visualizing the 'network_nodes_clustered.csv' and 'network_edges.csv' in external tools like Gephi or Cytoscape for more advanced layouts and visual analysis.")
        report_lines.append("  - Analyzing the distribution of centrality measures within and across clusters.")
        report_lines.append("  - Investigating the biological implications of distinct connected components and clusters, correlating them with taxonomic groups or ecological niches.")
        report_lines.append("  - Testing different similarity metrics or thresholds to observe changes in network topology and clustering.")
        report_lines.append("  - Deeper examination of the organisms within each cluster to confirm the biological relevance of the representative species.")

    else:
        report_lines.append("The network could not be built or is too sparse (e.g., all similarities below threshold).")
        report_lines.append("Consider lowering the `similarity_threshold` or checking your input data for sufficient variation.")
else:
    report_lines.append("No sequences were processed to build the network.")
    report_lines.append("Please check the input FASTA file and the feature extraction process.")


final_report = "\n".join(report_lines)
print(final_report)

with open('06_network_analysis_report.txt', 'w') as f:
    f.write(final_report)
print("\n✓ Network analysis complete! Report saved to 06_network_analysis_report.txt")
print("="*70)

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Load data
nodes_df = pd.read_csv('network_nodes_clustered.csv')
edges_df = pd.read_csv('network_edges.csv')

# Create Network Graph
G = nx.Graph()

# Add nodes with attributes
for _, row in nodes_df.iterrows():
    G.add_node(row['node_id'], cluster=row['cluster_id'], organism=row['organism'])

# Add edges with similarity weights
for _, row in edges_df.iterrows():
    G.add_edge(row['source'], row['target'], weight=row['weight'])

# Position nodes with a force-directed layout
pos = nx.spring_layout(G, k=0.15, iterations=20, seed=42)

# Assign distinct colors for each cluster
clusters = nodes_df['cluster_id'].unique()
colors = plt.cm.tab20(range(len(clusters)))
color_map = {cluster: colors[i] for i, cluster in enumerate(clusters)}

# Determine representative species per cluster
representatives = nodes_df.groupby('cluster_id').first().reset_index()
legend_labels = {row['cluster_id']: f"Cluster {row['cluster_id']}: {row['organism']}" 
                 for _, row in representatives.iterrows()}

# Plotting the Network
plt.figure(figsize=(16, 12))

for cluster in clusters:
    node_list = [node for node in G.nodes if G.nodes[node]['cluster'] == cluster]
    nx.draw_networkx_nodes(G, pos,
                           nodelist=node_list,
                           node_size=50,
                           node_color=[color_map[cluster]],
                           label=legend_labels[cluster])

# Draw edges with transparency based on similarity weight
weights = [G[u][v]['weight'] for u, v in G.edges()]
nx.draw_networkx_edges(G, pos, alpha=0.2, width=[w * 0.5 for w in weights])

# Plot formatting
plt.axis('off')
plt.title('Mitochondrial Genome Similarity Network with Representative Species')
plt.legend(title='Clusters & Representative Species', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')
plt.tight_layout()
plt.show()

# OPTIONAL: Additional Plot Suggested – Degree Centrality Distribution
degree_centrality = nx.degree_centrality(G)
centrality_values = list(degree_centrality.values())

plt.figure(figsize=(10, 6))
plt.hist(centrality_values, bins=30, color='skyblue', edgecolor='black')
plt.title('Distribution of Degree Centrality in Mitochondrial Genome Network')
plt.xlabel('Degree Centrality')
plt.ylabel('Frequency')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Define cricket species by common known genera (Acheta, Gryllus, etc.)
cricket_genera = ['Acheta', 'Gryllus', 'Tachycines', 'Gryllotalpa', 'Gryllodes', 'Teleogryllus']

# Identify cricket nodes
cricket_nodes = [row['node_id'] for _, row in nodes_df.iterrows() if any(genus in row['organism'] for genus in cricket_genera)]

# Plot with cricket nodes highlighted
plt.figure(figsize=(18, 12))

# Draw all nodes first
nx.draw_networkx_nodes(G, pos,
                       nodelist=G.nodes(),
                       node_size=30,
                       node_color='gray',
                       alpha=0.3)

# Highlight cricket nodes
nx.draw_networkx_nodes(G, pos,
                       nodelist=cricket_nodes,
                       node_size=80,
                       node_color='red',
                       label='Cricket Species')

# Draw edges
nx.draw_networkx_edges(G, pos, alpha=0.2, width=[G[u][v]['weight']*0.5 for u, v in G.edges()])

# Legend with clusters
for cluster in clusters:
    node_list = [node for node in G.nodes if G.nodes[node]['cluster'] == cluster and node not in cricket_nodes]
    nx.draw_networkx_nodes(G, pos,
                           nodelist=node_list,
                           node_size=30,
                           node_color=[color_map[cluster]],
                           alpha=0.5,
                           label=f'Cluster {cluster}')

# Formatting
plt.axis('off')
plt.title('Mitochondrial Genome Similarity Network with Cricket Species Highlighted')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()


In [ ]:
# Determine representative species for each cluster (first species encountered for simplicity)
representatives = nodes_df.groupby('cluster_id').first().reset_index()

# Plot with representative species names
plt.figure(figsize=(18, 12))

for cluster in clusters:
    node_list = [node for node in G.nodes if G.nodes[node]['cluster'] == cluster]
    nx.draw_networkx_nodes(G, pos,
                           nodelist=node_list,
                           node_size=50,
                           node_color=[color_map[cluster]],
                           label=f'Cluster {cluster}')

# Draw edges
nx.draw_networkx_edges(G, pos, alpha=0.3, width=[G[u][v]['weight']*0.5 for u, v in G.edges()])

# Annotate representative species
for _, row in representatives.iterrows():
    node_id = row['node_id']
    organism = row['organism']
    if node_id in pos:
        x, y = pos[node_id]
        plt.text(x, y, organism, fontsize=9, fontweight='bold', ha='center', va='center',
                 bbox=dict(facecolor='white', alpha=0.7, edgecolor='black', boxstyle='round,pad=0.2'))

# Formatting
plt.axis('off')
plt.title('Mitochondrial Genome Similarity Network with Representative Species')
plt.legend(title='Clusters', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()


In [ ]:
# List species within Cluster 0
cluster_0_species = nodes_df[nodes_df['cluster_id'] == 0]['organism'].unique()

cluster_0_species


Most of the species listed in Cluster 0 are indeed crickets (Family Gryllidae) or closely related species within Ensifera (long-horned Orthoptera). However, not all belong strictly to the family Gryllidae.

Clarified Classification:
Gryllidae (true crickets):

Acheta domesticus (house cricket)

Gryllus bimaculatus (two-spotted cricket)

Gryllus lineaticeps

Gryllus veletis

Loxoblemmus doenitzi

Tarbinskiellus portentosus

Tarbinskiellus sp.

Teleogryllus emma

Teleogryllus infernalis

Teleogryllus mitratus

Teleogryllus occipitalis

Teleogryllus oceanicus

Turanogryllus eous

Velarifictorus hemelytrus

Xenogryllus lamottei

Xenogryllus maniema

Xenogryllus marmoratus

Truljalia hibinonis

Gryllacrididae (raspy crickets):

Phryganogryllacris superangulata (not a true cricket, but closely related)

Other related genera:

Nisitrus vittatus (family Gryllidae or related subfamilies; considered cricket-like)

Pseudolebinthus sp. (Gryllidae or closely related cricket genus)

Sclerogryllus punctatus (Gryllidae or closely related cricket genus)

Euscyrtinae sp. (generally classified within Gryllidae or cricket-related group)

Summary:
The cluster predominantly includes true cricket species (family Gryllidae), with a few related genera such as Phryganogryllacris from Gryllacrididae, indicating that this cluster broadly represents cricket-like Orthoptera, particularly within the Ensifera lineage.

In [ ]:
# Extract species lists for specified clusters
clusters_of_interest = [0, 3, 5, 7, 24]
cluster_species_lists = {cluster: nodes_df[nodes_df['cluster_id'] == cluster]['organism'].unique().tolist()
                         for cluster in clusters_of_interest}

# Display clearly in a structured dictionary format
cluster_species_lists


Cluster 0 (Primarily Gryllidae)
Acheta domesticus, Euscyrtinae sp., Gryllus bimaculatus, Gryllus lineaticeps, Gryllus veletis, Loxoblemmus doenitzi, Nisitrus vittatus, Phryganogryllacris superangulata, Pseudolebinthus sp., Sclerogryllus punctatus, Tarbinskiellus portentosus, Tpa_asm: Tarbinskiellus, Tarbinskiellus sp., Teleogryllus emma, Teleogryllus infernalis, Teleogryllus mitratus, Teleogryllus occipitalis, Teleogryllus oceanicus, Truljalia hibinonis, Turanogryllus eous, Velarifictorus hemelytrus, Xenogryllus lamottei, Xenogryllus maniema, Xenogryllus marmoratus

Cluster 3 (Mixed, primarily Ensifera)
Anabropsis spp., Pteranabropsis spp., Cacoplistes rogenhoferi, Cardiodactylus muiri, Gryllotalpa spp., Dianemobius spp., Diestramima spp., Stenopelmatus spp., and many other related genera.

Cluster 5 (Cave cricket, Rhaphidophoridae)
Tachycines zorzini

Cluster 7 (Mixed, Ensifera)
Anabrus simplex, Alloxiphidiopsis emarginata, Gryllotalpa spp., Gryllodes spp., Phaneroptera spp., Xiphidiopsis spp., Xizicus spp., and numerous other diverse genera.

Cluster 24 (Cave cricket, Rhaphidophoridae)
Tachycines shuangcha

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Load data
nodes_df = pd.read_csv('network_nodes_clustered.csv')
edges_df = pd.read_csv('network_edges.csv')

# Create overall graph
G = nx.Graph()

# Add nodes with cluster info
for _, row in nodes_df.iterrows():
    G.add_node(row['node_id'], cluster=row['cluster_id'], organism=row['organism'])

# Add edges with similarity scores
for _, row in edges_df.iterrows():
    G.add_edge(row['source'], row['target'], weight=row['weight'])

# Select clusters to plot separately
selected_clusters = [0, 3, 5, 7, 24]

# Plot separate network graphs for each cluster
for cluster_id in selected_clusters:
    # Extract nodes belonging to the current cluster
    cluster_nodes = [node for node, data in G.nodes(data=True) if data['cluster'] == cluster_id]
    
    # Create subgraph for the current cluster
    subG = G.subgraph(cluster_nodes)
    
    # Position nodes
    pos = nx.spring_layout(subG, k=0.15, iterations=20, seed=42)
    
    # Plotting
    plt.figure(figsize=(12, 9))
    nx.draw_networkx_nodes(subG, pos, node_size=100, node_color='skyblue', alpha=0.9)
    nx.draw_networkx_edges(subG, pos, alpha=0.5, width=[subG[u][v]['weight']*0.5 for u, v in subG.edges()])
    nx.draw_networkx_labels(subG, pos, labels={node: data['organism'] for node, data in subG.nodes(data=True)}, 
                            font_size=8, font_color='darkblue')
    
    # Title and formatting
    plt.title(f'Network Visualization of Cluster {cluster_id}', fontsize=16)
    plt.axis('off')
    plt.tight_layout()
    plt.show()


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Load data
nodes_df = pd.read_csv('network_nodes_clustered.csv')
edges_df = pd.read_csv('network_edges.csv')

# Create overall graph
G = nx.Graph()

# Add nodes with attributes
for _, row in nodes_df.iterrows():
    G.add_node(row['node_id'], cluster=row['cluster_id'], organism=row['organism'])

# Add edges with similarity weights
for _, row in edges_df.iterrows():
    G.add_edge(row['source'], row['target'], weight=row['weight'])

# Define cricket genera to highlight
cricket_genera = ['Acheta', 'Gryllus', 'Tachycines', 'Gryllotalpa', 'Gryllodes', 'Teleogryllus', 
                  'Velarifictorus', 'Xenogryllus', 'Truljalia', 'Tarbinskiellus', 'Loxoblemmus']

def is_cricket(organism):
    return any(genus in organism for genus in cricket_genera)

# Select clusters to visualize
selected_clusters = [0, 3, 5, 7, 24]

for cluster_id in selected_clusters:
    # Nodes in the current cluster
    cluster_nodes = [node for node, data in G.nodes(data=True) if data['cluster'] == cluster_id]
    
    # Subgraph
    subG = G.subgraph(cluster_nodes)
    
    # Position nodes
    pos = nx.spring_layout(subG, k=0.15, iterations=20, seed=42)
    
    # Identify cricket nodes
    cricket_nodes = [node for node, data in subG.nodes(data=True) if is_cricket(data['organism'])]
    non_cricket_nodes = [node for node in subG.nodes if node not in cricket_nodes]
    
    # Plotting
    plt.figure(figsize=(14, 10))
    
    # Non-cricket nodes
    nx.draw_networkx_nodes(subG, pos, nodelist=non_cricket_nodes, 
                           node_size=150, node_color='lightgray', alpha=0.8, label='Other Species')
    
    # Cricket nodes
    nx.draw_networkx_nodes(subG, pos, nodelist=cricket_nodes, 
                           node_size=200, node_color='red', alpha=0.9, label='Cricket Species')
    
    # Edges
    nx.draw_networkx_edges(subG, pos, alpha=0.4, width=[subG[u][v]['weight']*0.5 for u, v in subG.edges()])
    
    # Species labels
    nx.draw_networkx_labels(subG, pos, labels={node: subG.nodes[node]['organism'] for node in subG.nodes()},
                            font_size=8, font_color='black')
    
    # Formatting
    plt.title(f'Network of Cluster {cluster_id} with Highlighted Cricket Species', fontsize=16)
    plt.legend(scatterpoints=1, fontsize=12)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
